### Load MNIST 

In [38]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28*28)
x_test = x_test.reshape(-1, 28*28)

In [39]:
from sklearn.model_selection import train_test_split

train_indxes_0to4 = y_train<5
train_x_0to4 = x_train[train_indxes_0to4][:10000]
train_y_0to4 = y_train[train_indxes_0to4][:10000]

test_indxes_0to4 = y_test<5
test_x_0to4 = x_test[test_indxes_0to4]
test_y_0to4 = y_test[test_indxes_0to4]



(train_x_0to4, val_x_0to4, train_y_0to4, val_y_0to4) = \
    train_test_split(train_x_0to4, train_y_0to4, test_size=0.1)
    

### Some Utility Functions

In [2]:
import tensorflow as tf
from tensorflow.contrib.layers import variance_scaling_initializer as he_initializer
from tensorflow.nn import sparse_softmax_cross_entropy_with_logits as softmax_xentropy
from tensorflow.layers import dense
import numpy as np


def get_leaky_relu(alpha):
    return lambda z, name=None: tf.maximum(alpha*z,z, name=name)
    

def get_connected_layers(x, n_hidden_layers, n_neurons, n_ouputs, activation=tf.nn.elu,
                                   batch_norm_momentum=None, dropout_rate=None):
    

    initializer = he_initializer()
    
    with tf.name_scope("DNN"):
        inputs = x
        for l in range(n_hidden_layers):
            if dropout_rate is not None:
                ## this function will set inputs to zero with dropout rate probability
                ## and divides remaining inputs with dropout rate
                inputs = tf.layers.dropout(inputs, dropout_rate, training=self._is_training, 
                                  name=("dropout%d"%l))
                
            inputs = tf.layers.dense(inputs, n_neurons, kernel_initializer=initializer,
                           name="hidden%d"%(l+1), activation=activation)
            
            if batch_norm_momentum is not None:
                inputs = tf.layers.batch_normalization(inputs, momentum=batch_norm_momentum,
                                training=self._is_training)
            
            inputs = activation(inputs, name="hiden%d_out"%(l+1))
            
        output = tf.layers.dense(inputs, n_ouputs, name="output")
        
    return output
        


def get_softmax_xentropy_loss(logits,y):
    with tf.name_scope("loss"):
        xentropy = softmax_xentropy(labels=y, logits=logits)
        return tf.reduce_mean(xentropy)

def get_optimizer_op(optimizer, loss, learning_rate=0.01):
    with tf.name_scope("train"):
        optimizer =  optimizer(learning_rate=learning_rate)
        optimizer_op = optimizer.minimize(loss)
    return optimizer_op

def get_validation_score(logits,y):
    with tf.name_scope("validation"):
        preds = tf.nn.in_top_k(logits,y,1)
        return tf.reduce_mean(tf.cast(preds, dtype=np.float32))
    
def get_batch(x,y,batch_size):
    n_batches = len(y)//batch_size + 1
    for i in range(n_batches):
        indxes = np.random.choice(len(y), size=batch_size, replace=False)
        yield x[indxes], y[indxes]

    

/Users/devbhadurkhadka/.pyenv/versions/anaconda3-5.2.0/envs/scikit_practice/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Classifier Class

In [15]:
from sklearn.base import BaseEstimator, TransformerMixin

class DNN_Classifier(BaseEstimator, TransformerMixin):
    def __init__(self, n_hidden_layers=None, n_neurons=None, n_outputs=None, 
                 activation=tf.nn.elu, optimizer=tf.train.AdamOptimizer,  learning_rate=0.01, 
                 batch_norm_momentum=None, batch_size=50, dropout_rate=None):
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.learning_rate = learning_rate
        self.activation = activation
        self.optimizer = optimizer
        self.batch_norm_momentum = batch_norm_momentum
        self.batch_size = batch_size
        self.dropout_rate = dropout_rate
        self.n_outputs = n_outputs
        self._session = None
        
        
    def _create_graph(self):                      
        
        tf.reset_default_graph()
        self._graph = tf.Graph()
        
        with self._graph.as_default():
        
            self._x = tf.placeholder(shape=(None, 28*28), dtype=np.float32,name="x")
            self._y = tf.placeholder(shape=(None), dtype=np.int32,name="y")

            self._is_training = tf.placeholder(shape=(), dtype=np.bool, name="is_training")


            self._dnn = get_connected_layers(self._x, self.n_hidden_layers, self.n_neurons, 
                                       self.n_outputs, activation=self.activation, 
                                       batch_norm_momentum=self.batch_norm_momentum, 
                                       dropout_rate=self.dropout_rate)
            self._loss = get_softmax_xentropy_loss(self._dnn, self._y)
            self._optimizer_op = get_optimizer_op(self.optimizer, self._loss, self.learning_rate)
            self._validation_score = get_validation_score(self._dnn, self._y)

            self._y_proba = tf.nn.softmax(self._dnn)

            self._batch_norm_update_ops = self._graph.get_collection(tf.GraphKeys.UPDATE_OPS)
            
            self._saver = tf.train.Saver()
            self._init = tf.global_variables_initializer()
            
        
    def _save_params(self):
        with self._graph.as_default():
            global_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
            
        vars_n_values = {global_var.op.name:value for global_var, value in \
                 zip(global_vars,self._session.run(global_vars))}
        self._saved_params =  vars_n_values
        
    
    def _restore_params(self):
        var_names = list(self._saved_params.keys())
        
        ## get assign operations for all variables
        assign_ops = {var_name:self._graph.get_operation_by_name("%s/Assign"%var_name) 
                      for var_name in var_names}
        ## get initialization values of all variables
        init_values = {var_name: assign_op.inputs[1]  for var_name, assign_op 
                       in assign_ops.items()}
        
        ## get feed_dict for all values
        feed_dict = {init_values[var_name]:self._saved_params[var_name] 
                     for var_name in var_names}
        
        
        self._session.run(assign_ops, feed_dict=feed_dict)
        
    
    def fit(self,x,y,x_val,y_val):
        n_epoches = 50
        max_epoches_wo_progress = 10
        
        self._create_graph()
        
        best_score=0
        best_epoch=0
        if self._session: self._session.close()
        with tf.Session(graph=self._graph).as_default() as sess:
            self._session = sess
            
            sess.run(self._init)
            for epoch in range(n_epoches):
                for batch_x, batch_y in get_batch(x,y,self.batch_size):
                    ops = [self._optimizer_op]
                    if self._batch_norm_update_ops is not None:
                        ops.append(self._batch_norm_update_ops)
                   
                    sess.run(ops , feed_dict={self._x:batch_x, self._y:batch_y, 
                                           self._is_training:True})
                
                
                
                score = sess.run(self._validation_score, 
                                 feed_dict={self._x:x_val, self._y:y_val})
                if epoch%50 == 0:
                    print("epoch %d, score %f"%(epoch, score))
                
                if score > best_score:
                    best_score = score
                    best_epoch = epoch
                    self._save_params()
                elif (epoch - best_epoch)>max_epoches_wo_progress:
                    print("No progress for %d epoches."%max_epoches_wo_progress)
                    break
                
            self._restore_params()
            print("Reverting back to epoch %d \
                    with %f score" %(best_epoch, best_score))
            self._score = best_score 
            return self
            
                    
    
    def predict_proba(self,x):
        if self._session is None:
            raise sklearn.exceptions.NotFittedError("%s is not fitted yet" \
                                                    %self.__class__.__name__)
        
        return self._session.run(self._y_proba, feed_dict={self._x:x, self._is_training:False})
            
    
    def predict(self,x):
        return np.argmax(self.predict_proba(x), axis=1)
    
    def score(self, x_val=None, y_val=None):
        
        score=self._session.run(self._validation_score, 
                             feed_dict={self._x:x_val, self._y:y_val})
        print("validation score: %f", score)
        return score
    
    def _get_save_path(self, name):
        return "checkpoints/%s"%name
    
    def save(self,name):
        self._saver.save(self._session, self._get_save_path(name))
    
    def restore(self, name):
        tf.train.import_meta_graph()
    

### Fitting Classifier

In [16]:
classifier = DNN_Classifier(2, 20, 5, activation=get_leaky_relu(0.01))
classifier.fit(train_x_0to4, train_y_0to4, val_x_0to4, val_y_0to4)

epoch 0, score 0.854000
No progress for 10 epoches.
Reverting back to epoch 37                     with 0.968000 score


DNN_Classifier(activation=<function get_leaky_relu.<locals>.<lambda> at 0x1c34758b70>,
        batch_norm_momentum=None, batch_size=50, dropout_rate=None,
        learning_rate=0.01, n_hidden_layers=2, n_neurons=20, n_outputs=5,
        optimizer=<class 'tensorflow.python.training.adam.AdamOptimizer'>)

### Saving and Restoring

#### Restore only works when we know the constructor parameters

In [7]:
classifier.save("test_save")

In [10]:
classifier2 = DNN_Classifier(2, 20, 5, activation=get_leaky_relu(0.01))
classifier2.restore("test_save")

W0822 17:15:45.995651 4545570240 deprecation.py:323] From /Users/devbhadurkhadka/.pyenv/versions/anaconda3-5.2.0/envs/scikit_practice/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [11]:
classifier2.score(val_x_0to4, val_y_0to4)

validation score: %f 0.961


0.961

#### Not working like this
- Because we need to create graph for restoring it and for creating graph we need constructor parameters

In [13]:
classifier2 = DNN_Classifier()
classifier2.restore("test_save")

TypeError: 'NoneType' object cannot be interpreted as an integer

### joblib is also not working 

In [17]:
from sklearn.externals import joblib

path = "checkpoints/save_test_joblib.chk"
joblib.dump(classifier, path)

PicklingError: Can't pickle <function get_leaky_relu.<locals>.<lambda> at 0x1c34758b70>: it's not found as __main__.get_leaky_relu.<locals>.<lambda>

### dill also not working

In [18]:
import dill
path = "checkpoints/save_test_dill.chk"
with open(path, "wb") as fl:
    dill.dump(classifier, fl)

TypeError: can't pickle SwigPyObject objects

In [1]:
import tensorflow as tf
import numpy as np

with tf.Graph().as_default() as graph:
    var_x = tf.Variable(0, shape=(), dtype=np.int16, name="x")
    var_x_ph = tf.placeholder(shape=(), name="x_ph", dtype=np.int16)
    assign_X = tf.assign(var_x, var_x_ph, name="assign_x")
    
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(assign_X, feed_dict={var_x_ph:32})
    print(var_x.eval())
    
    saver.save(sess, save_path="checkpoints/test_saver")



/Users/devbhadurkhadka/.pyenv/versions/anaconda3-5.2.0/envs/scikit_practice/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


32


In [44]:
tf.reset_default_graph()
imported_meta = tf.train.import_meta_graph("tf_checkpoints/Mnist-0to4-best1.ckpt.meta")

graph = tf.get_default_graph()

gvars = tf.get_collection(key="trainable_variables")
graph.get_operations()
loss = graph.get_operation_by_name("loss/Mean")
with tf.Session() as sess:
    imported_meta.restore(sess, "tf_checkpoints/Mnist-0to4-best1.ckpt")
    out = sess.run(loss, feed_dict={"x:0": val_x_0to4, "y:0": val_y_0to4 })



None


In [47]:
import tensorflow.keras as keras
keras.layers.D

<module 'tensorflow.keras' from '/Users/devbhadurkhadka/.pyenv/versions/anaconda3-5.2.0/envs/scikit_practice/lib/python3.6/site-packages/tensorflow/python/keras/api/_v1/keras/__init__.py'>

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class CustomTransformer(BaseEstimator, TransformerMixin):
    def fit(self,X,y):
        self.svc = SVC() #initialize your svc here
        return self
    
    def transform(self,X,y=None):
        return self.svc.predict(X)
    
    